In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats


In [5]:
#Creamos el dataframe a partir del set de datos y les asigno un tipo acorde a las descripciones
df_train = pd.read_csv('data/train.csv', dtype={'id':'int', 'titulo':'object','descripcion':'object',\
                                           'tipodepropiedad':'category', 'direccion':'object',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float', 'lat':'float', 'lng':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool',\
                                           'precio':'float'}, parse_dates=['fecha'])
df_submit = pd.read_csv('data/test.csv', dtype={'id':'int', 'titulo':'object','descripcion':'object',\
                                           'tipodepropiedad':'category', 'direccion':'object',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float', 'lat':'float', 'lng':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool',\
                                           'precio':'float'}, parse_dates=['fecha'])

In [6]:
df_train.shape

(240000, 23)

In [7]:
df_submit.shape

(60000, 22)

In [8]:
df_train.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [9]:
df_submit.isnull().sum()

id                                0
titulo                         1378
descripcion                     401
tipodepropiedad                   7
direccion                     13191
ciudad                           83
provincia                        42
antiguedad                    10714
habitaciones                   5628
garages                        9323
banos                          6554
metroscubiertos                4299
metrostotales                 12655
idzona                         7179
lat                           30695
lng                           30695
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
dtype: int64

In [10]:
df_train.describe()

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,precio
count,240000.000000,196445.000000,217529.000000,202235.000000,213779.000000,222600.000000,188533.000000,2.113790e+05,116512.000000,116512.000000,2.400000e+05
mean,149969.382092,8.116114,2.902326,1.546874,2.132417,174.016774,176.765145,2.423468e+06,20.696818,-99.509221,2.530838e+06
std,86634.579744,9.553830,0.896894,0.853507,0.912546,98.152950,94.427328,1.056794e+07,3.137884,9.744143,2.152552e+06
min,1.000000,0.000000,1.000000,0.000000,1.000000,15.000000,15.000000,2.200000e+01,-100.886679,-125.859375,3.100000e+05
25%,74930.750000,0.000000,2.000000,1.000000,1.000000,90.000000,102.000000,2.489000e+04,19.359846,-100.977908,9.527725e+05
50%,149875.500000,5.000000,3.000000,2.000000,2.000000,153.000000,155.000000,5.638300e+04,19.543273,-99.240387,1.850000e+06
75%,225016.500000,10.000000,3.000000,2.000000,3.000000,240.000000,238.000000,8.783800e+04,20.740005,-99.134777,3.390000e+06
max,299999.000000,80.000000,10.000000,3.000000,4.000000,439.000000,439.000000,5.000400e+07,83.026219,121.036000,1.252500e+07


In [11]:
df_train.corr()['precio'].sort_values(ascending=False)

precio                        1.000000
metroscubiertos               0.629187
banos                         0.534060
metrostotales                 0.514411
garages                       0.348543
habitaciones                  0.251014
gimnasio                      0.153898
usosmultiples                 0.145407
piscina                       0.112712
idzona                        0.073644
antiguedad                    0.073097
lng                           0.052578
centroscomercialescercanos    0.028127
escuelascercanas              0.001187
id                           -0.000889
lat                          -0.157514
Name: precio, dtype: float64

Lo primero a analizar es ver que puedo hacer con respecto a los datos faltantes, si bien hay algunos modelos que los NaN los manejan (por ejemplo XGBoost), es importante encargarse de ellos.
Tambien voy a limpiar un poco el set de datos borrando aquellas publicaciones que son test/ duplicados

In [12]:
#Elimino duplicados (si es que los hay)
df_train.drop_duplicates(keep='first').shape

(240000, 23)

In [13]:
#Al parecer no hay duplicados

In [14]:
# Con respecto a latitud y longitud, se podrian reemplazar los valores por
# el promedio de la latitud/longitud de esa ciudad, y en caso de no haber datos 
# de la ciudad, reemplazar por el promedio de la latitud/longitud
# de la provincia correspondiente

# Dado que el promedio de la latitud/longitud no varia en gran grado,
# y estan relacionadas con la provincia y ciudad, las voy a droppear.

df_train.drop(['lat','lng'], axis=1, inplace=True)

#idem submit

df_submit.drop(['lat','lng'], axis=1, inplace=True)

In [15]:
#Titulo y direccion tampoco voy a utilizarlas

df_train.drop(['titulo','direccion'], axis=1, inplace=True)

#idem submit

df_submit.drop(['titulo','direccion'], axis=1, inplace=True)

In [16]:
#Voy a definir una funcion que analice si una publicacion es test, en base a
#su descripcion, y despues chequeo a mano cuales son test y cuales no.

In [17]:
def verSiEsTest(x):
    if pd.isna(x): return False
    lista_palabras = x.split()
    palabras = ['test','Test','TEST','prueba','PRUEBA','Prueba']
    for p in palabras:
        if p in lista_palabras:
            return True
    return False

In [18]:
df_train['test'] = df_train['descripcion'].apply(lambda x: verSiEsTest(x))

In [19]:
df_test = df_train[df_train['test'] == True]

In [20]:
df_test.head()

,id,descripcion,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,test
18685,262385,prueba aviso 1 prueba aviso 1 prueba aviso 1 p...,Casa,Cosío,Aguascalientes,5.0,3.0,1.0,2.0,124.0,126.0,517.0,2016-08-05,True,False,False,False,False,1000000.0,True
24919,293504,lista para ocupar. con todas las comodidades. ...,Casa,Mérida,Yucatán,10.0,3.0,2.0,2.0,127.0,176.0,369425.0,2016-08-27,False,False,False,True,True,900000.0,True
25706,162718,extraordinario departamento en providencia. el...,Apartamento,Guadalajara,Jalisco,NaN,3.0,2.0,3.0,181.0,181.0,47484.0,2016-04-17,True,True,True,True,True,5789000.0,True
32751,151081,"se vende casa que cuenta con sala, comedor, co...",Casa,Ixtapaluca,Edo. de México,5.0,3.0,2.0,3.0,192.0,180.0,59192.0,2012-09-09,False,False,False,True,True,1400000.0,True
33883,299384,disfruta tu vida con la ubicación ideal.\r\nce...,Apartamento,Benito Juárez,Distrito Federal,NaN,3.0,2.0,2.0,99.0,99.0,23569.0,2016-12-17,False,False,False,False,False,5046000.0,True


In [21]:
#Lista de ids de publicaciones que son tests, armada a mano mirando las publicaciones
lista_id_test = [139830,89338,248160,198221,149285,215051,53598,225640,273393,154112,171296,262385]

In [22]:
df_train = df_train[~(df_train['id'].isin(lista_id_test))]

#Tambien dropeo las columnas que ya no utilizo
df_train.drop(['test','descripcion'],axis=1,inplace=True)
df_train.set_index('id',inplace=True)

#idem submit

df_submit.drop(['descripcion'],axis=1,inplace=True)
df_submit.set_index('id',inplace=True)

In [23]:
#En cuanto a la fecha, vimos que no era un factor determinante en el tp1, pero
#de todas formas la voy a conservar (en 3 columas por separado)

df_train['año'] = df_train['fecha'].dt.year
df_train['mes'] = df_train['fecha'].dt.month
df_train['dia'] = df_train['fecha'].dt.day

df_train.drop('fecha',axis=1,inplace=True)

#idem submit

df_submit['año'] = df_submit['fecha'].dt.year
df_submit['mes'] = df_submit['fecha'].dt.month
df_submit['dia'] = df_submit['fecha'].dt.day

df_submit.drop('fecha',axis=1,inplace=True)


In [24]:
df_train.head()

,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,año,mes,dia
id,,,,,,,,,,,,,,,,,,,
254099,Apartamento,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,False,False,False,False,False,2273000.0,2015,8,23
53461,Casa en condominio,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,False,False,False,True,True,3600000.0,2013,6,28
247984,Casa,Tonalá,Jalisco,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,False,False,False,False,False,1200000.0,2015,10,17
209067,Casa,Zinacantepec,Edo. de México,1.0,2.0,1.0,1.0,63.0,67.0,53666.0,False,False,False,True,True,650000.0,2012,3,9
185997,Apartamento,Zapopan,Jalisco,10.0,2.0,1.0,1.0,95.0,95.0,47835.0,False,False,False,False,False,1150000.0,2016,6,7


En cuanto a los Outliers, voy a dropearlos utilizando el Zcore (solo para valores
 numericos)

In [25]:
df_train.shape

(239988, 19)

In [26]:
df_submit.shape

(60000, 18)

In [27]:
df_train.shape

(239988, 19)

In [28]:
#Me dropeaba todas las filas, raro, lo dejo para despues

In [29]:
df_train.isnull().sum()

tipodepropiedad                  46
ciudad                          372
provincia                       155
antiguedad                    43553
habitaciones                  22467
garages                       37763
banos                         26218
metroscubiertos               17400
metrostotales                 51463
idzona                        28621
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
precio                            0
año                               0
mes                               0
dia                               0
dtype: int64

In [30]:
df_submit.isnull().sum()

tipodepropiedad                   7
ciudad                           83
provincia                        42
antiguedad                    10714
habitaciones                   5628
garages                        9323
banos                          6554
metroscubiertos                4299
metrostotales                 12655
idzona                         7179
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
año                               0
mes                               0
dia                               0
dtype: int64

In [31]:
#Dado que el tipo de propiedad, la ciudad, y la provincia, eran los factores que mas
#influian en el precio de una propiedad, voy a droppear las filas nulos de estos.
df_train = df_train.dropna(subset=['tipodepropiedad','ciudad','provincia'])

In [32]:
df_train.to_csv('data/train_limpio.csv')
df_submit.to_csv('data/submit_limpio.csv')

In [33]:
#Si quiero usar el dataframe limpio para modelos que no aceptan NaN, voy
#a utilizar el imputer, pero eso se hace por separado en train y test
